## Facebook Poverty Estimation: Within Country

In [1]:
import os, datetime
import numpy as np
import pandas as pd
#import geopandas as gpd
import json
#import rasterio
#from rasterio.plot import show
import matplotlib.pyplot as plt

from sklearn.preprocessing import KBinsDiscretizer, StandardScaler, normalize
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import Ridge

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (BaggingClassifier, AdaBoostClassifier,
                              AdaBoostRegressor,
                              GradientBoostingClassifier, RandomForestClassifier,
                              RandomForestRegressor,
                             BaggingRegressor, GradientBoostingRegressor)
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (accuracy_score, precision_score, 
                             recall_score, classification_report,
                            r2_score, mean_absolute_error, mean_squared_error)

from joblib import dump, load

import logging, os 

import grid_params as grids

In [2]:
data_dir = "/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satellites/Data"
SURVEY_NAME = "DHS"

OUT_DIR = os.path.join(data_dir, SURVEY_NAME, 'FinalData', 'results')

## Load/Prep Data

In [3]:
survey_df = pd.read_csv(os.path.join(data_dir, SURVEY_NAME, "FinalData", "Individual Datasets", "survey_socioeconomic.csv"))

/Users/robmarty/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (27,31,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
viirs_df = pd.read_csv(os.path.join(data_dir, 
                            SURVEY_NAME, 
                            'FinalData', 
                            'Individual Datasets',
                           'survey_viirs.csv'))

l8_df = pd.read_csv(os.path.join(data_dir, 
                            SURVEY_NAME, 
                            'FinalData', 
                            'Individual Datasets',
                           'survey_l8.csv'))

sat_df = l8_df.merge(viirs_df, on = "uid")

sat_df.viirs_avg_rad = np.log(sat_df.viirs_avg_rad+1)

In [5]:
fb_df = pd.read_csv(os.path.join(data_dir, SURVEY_NAME, 'FinalData', 'Individual Datasets', 'facebook_marketing_dau_mau.csv'))
fb_prop_df = pd.read_csv(os.path.join(data_dir, SURVEY_NAME, 'FinalData', 'Individual Datasets', 'facebook_marketing_dau_mau_prop.csv'))

In [6]:
fb_df = fb_df[['uid', 'estimate_mau_1']]

In [7]:
df = survey_df.merge(fb_df, on = 'uid')
df = df.merge(fb_prop_df, on = 'uid')
df = df.merge(sat_df, on = 'uid')
#df = df.sample(frac=1).reset_index(drop=True)

## Functions

In [8]:
def trainmodel_valresult(df, country, est_type, target, parameters, feature_type):
    # DESCRIPTION:
    # ARGS:
    # df: Dataframe
    # country: iso2
    # est_type: within_country or other_countries

    if est_type == 'within_country_cv':
        df_traintest = df[df.country_code == country]
        df_traintest.reset_index()
                
    # Initialize
    results_df = pd.DataFrame() # results iterating over params
    y_df = df_traintest[['uid', 'country_code']]
    y_df['y'] = df_traintest[target]
    y_df['target'] = target
    y_df['feature_type'] = feature_type
        
    model_i = 0
    for i in parameters['regressors']:
        for j in parameters[i]:
            
            pred_dict = {
                'regressor': i,
                'params': j,
                'country': country,
                'est_type': est_type,
                'target': target,
                'model_i': model_i,
                'feature_type': feature_type
             }

            fold = 0
            y_df_parami = pd.DataFrame()
            for split_id in df_traintest.within_country_fold.unique():
                
                # Separate into train and test
                df_train = df_traintest[df_traintest['within_country_fold'] != split_id]
                df_test = df_traintest[df_traintest['within_country_fold'] == split_id]
                
                if (feature_type == "fb_only"):
                    x_train = df_train.filter(regex='^estimate_mau_', axis=1)
                    x_test = df_test.filter(regex='^estimate_mau_', axis=1)
                    
                if (feature_type == "sat_only"):
                    x_train = df_train.filter(regex='^l8_|^viirs_avg_', axis=1)
                    x_test = df_test.filter(regex='^l8_|^viirs_avg_', axis=1)
                    
                if (feature_type == "all"):
                    x_train = df_train.filter(regex='^estimate_mau_|^l8_|^viirs_avg_', axis=1)
                    x_test = df_test.filter(regex='^estimate_mau_|^l8_|^viirs_avg_', axis=1)
                    
                y_train = df_train[target]
                y_test = df_test[target]

                # Normalize
                x_scaler = StandardScaler().fit(x_train)

                x_train = x_scaler.transform(x_train)
                x_test = x_scaler.transform(x_test)

                ### Initialize regressor, fit data, then append model to list
                regressor = eval(i)(**j)
                trained = regressor.fit(x_train, y_train)

                ### Results
                y_pred = trained.predict(x_test)
                
                pred_dict['r2_score_' + str(fold)] = r2_score(y_test, y_pred)
                
                y_dict_foldi = {
                    'uid': df_test.uid
                 }
                y_dict_foldi['y_' + str(model_i)] = y_pred
                
                y_df_foldi = pd.DataFrame.from_dict(y_dict_foldi)
                y_df_parami = y_df_parami.append(y_df_foldi, ignore_index=True)

                
                fold += 1

            y_df = y_df.merge(y_df_parami, on = 'uid', how = 'right')
            y_df.to_csv(os.path.join(OUT_DIR, 'country_withincv',
                               'withincv_predicted_values_' + 
                               country + '_' +
                               est_type + '_' +
                               target + 
                               '_fbonly.csv'))
                
            results_df = results_df.append(pred_dict, ignore_index=True)
            results_df.to_csv(os.path.join(OUT_DIR, 'country_withincv',
                                           'withincv_results_' + 
                                           country + '_' +
                                           est_type + '_' +
                                           target + 
                                           '_fbonly.csv'))
            model_i += 1
        
    return results_df, y_df

In [9]:
parameters = grids.GRID_REGRESS

In [10]:
results_all_df = pd.DataFrame()
y_all_df = pd.DataFrame()

for feature_type_i in ['fb_only', 'sat_only', 'all']:
    for est_type_i in ['within_country_cv']:
        for cc_i in df.country_code.unique():
            for target in ['asset_pca_1', 'wealth_index_score']:
                print(est_type_i + ' ' + cc_i)

                results_df_i, y_df_i = trainmodel_valresult(df, cc_i, est_type_i, target, parameters, feature_type_i)

                results_all_df = results_all_df.append(results_df_i, ignore_index=True)
                y_all_df = y_all_df.append(y_df_i, ignore_index=True)

within_country_cv BD


/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

within_country_cv BD


/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

within_country_cv IA


...

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

within_country_cv TL


/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

within_country_cv TL


/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [11]:
results_all_df.to_csv(os.path.join(OUT_DIR, 'results_fbonly_withincv.csv'))
y_all_df.to_csv(os.path.join(OUT_DIR, 'ypred_fbonly_withincv.csv'))